# BabyAGI User Guide

This notebook demonstrates how to implement [BabyAGI](https://github.com/yoheinakajima/babyagi/tree/main) by [Yohei Nakajima](https://twitter.com/yoheinakajima). BabyAGI is an AI agent that can generate and pretend to execute tasks based on a given objective, and you can find the origin notebook in [LangChain example](https://github.com/hwchase17/langchain/blob/master/docs/use_cases/autonomous_agents/baby_agi.ipynb).

This guide will help you understand the components to create your own recursive agents, And it will also show how to use GPTCache to cache the response. You can also try this example on [Google Colab](https://colab.research.google.com/drive/1WTvWIujioZtpwwVz7GGDzYhooul-rBeG?usp=sharing).

Although BabyAGI uses specific vectorstores/model providers (Pinecone, OpenAI), one of the benefits of implementing it with LangChain is that you can easily swap those out for different options. In this implementation we use a Milvus vector datavase (because it runs locally and is free).

 

## Go into GPTCache

Please [install gptcache](https://gptcache.readthedocs.io/en/latest/index.html#) first, then we can initialize the cache.There are two ways to initialize the cache, the first is to use the map cache (exact match cache) and the second is to use the DataBse cache (similar search cache), it is more recommended to use the second one, but you have to install the related requirements.

Before running the example, make sure the `OPENAI_API_KEY` environment variable is set by executing `echo $OPENAI_API_KEY`. If it is not already set, it can be set by using `export OPENAI_API_KEY=YOUR_API_KEY` on Unix/Linux/MacOS systems or `set OPENAI_API_KEY=YOUR_API_KEY` on Windows systems. And there is `get_content_func` for the cache settings:

In [1]:
# get the content(only question) form the prompt to cache
def get_content_func(data, **_):
    return data.get("prompt").split("Question")[-1]

### 1. Init for exact match cache

In [2]:
# from gptcache import cache
# cache.init(pre_embedding_func=get_content_func)
# cache.set_openai_key()

### 2. Init for similar match cache

When initializing gptcahe, the following four parameters are configured:

- `pre_embedding_func`: pre-processing before extracting feature vectors, it will use the `get_content_func` method
- `embedding_func`: the method to extract the text feature vector
- `data_manager`: DataManager for cache management
- `similarity_evaluation`: the evaluation method after the cache hit

The `data_manager` is used to audio feature vector, response text in the example, it takes [Milvus](https://milvus.io/docs) (please make sure it is started), you can also configure other vector storage, refer to [VectorBase API](https://gptcache.readthedocs.io/en/latest/references/manager.html#module-gptcache.manager.vector_data).

In [3]:
from gptcache import cache
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation


onnx = Onnx()
cache_base = CacheBase('sqlite')
vector_base = VectorBase('milvus', host='127.0.0.1', port='19530', dimension=onnx.dimension)
data_manager = get_data_manager(cache_base, vector_base)
cache.init(
    pre_embedding_func=get_content_func,
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
    )
cache.set_openai_key()

After initializing the cache, you can use the LangChain LLMs with `gptcache.adapter.langchain_models`. At this point **gptcache** will cache the answer, the only difference from the original example is to change `llm = OpenAI(temperature=0)` to `llm = LangChainLLMs(llm=OpenAI(temperature=0))`, which will be commented in the code block. And you can also set the `session` to set the session settings with `llm = LangChainLLMs(llm=OpenAI(temperature=0), session=session)`, more details refer to [session example](https://github.com/zilliztech/GPTCache/tree/main/examples#how-to-run-with-session).

Then you will find that it will be more fast when search the similar content, let's play with it.

## Install and Import Required Modules

In [4]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.experimental import BabyAGI

from gptcache.adapter.langchain_models import LangChainLLMs
from gptcache.session import Session
session = Session(name="baby_agi") # set session for LangChainLLMs

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [5]:
# from langchain.vectorstores import FAISS
from langchain.vectorstores import Milvus
from langchain.docstore import InMemoryDocstore

In [6]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
embedding_size = 1536
vectorstore = Milvus(embeddings_model,
                     collection_name="baby_agi",
                     connection_args={"host": "127.0.0.1", "port": "19530"},
                    )

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [7]:
OBJECTIVE = "Write a weather report for SF today"

In [8]:
# llm = OpenAI(temperature=0)
llm = LangChainLLMs(llm=OpenAI(temperature=0), session=session)

In [9]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)

In [10]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****



1. Check the weather forecast for San Francisco today
2. Make note of the temperature, humidity, wind speed, and other relevant weather conditions
3. Write a weather report summarizing the forecast
4. Check for any weather alerts or warnings
5. Share the report with the relevant stakeholders

*****TASK LIST*****

2: Check the current temperature in San Francisco
3: Check the current humidity in San Francisco
4: Check the current wind speed in San Francisco
5: Check for any weather alerts or warnings in San Francisco
6: Check the forecast for the next 24 hours in San Francisco
7: Check the forecast for the next 48 hours in San Francisco
8: Check the forecast for the next 72 hours in San Francisco
9: Check the forecast for the next week in San Francisco
10: Check the forecast for the next month in San Francisco
11: Check the forecast for the next 3 months in San Francisco
1: Writ

{'objective': 'Write a weather report for SF today'}